In [200]:
#pip install matplotlib
import matplotlib.pyplot as plt
#pip install numpy
import numpy as np
#pip install pandas
import pandas as pd
#pip install dataframe_image
import dataframe_image as dfi
import os
import time

from datetime import datetime, timedelta, date
from matplotlib import dates as mpl_dates
from ipywidgets.widgets import Label, Layout, Button, Text 
from dateutil.relativedelta import relativedelta

/* @author  Bart Withaar, Joan den Otter, Simon Balk
 * @date    26-5-2021
 * @brief   All the global variables which need to be declared
 * @param   none
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [201]:
#Reading the data from the csv file
data_from_csv = pd.read_csv('Dummy_data.CSV')
data_from_csv['Date'] = pd.to_datetime(data_from_csv.Date)
data_from_csv['Week'] = data_from_csv.Date.dt.isocalendar().week
data_from_csv['Month'] = data_from_csv.Date.dt.month
data_from_csv['Day'] = data_from_csv.Date.dt.day

#Setting the device_ID and time period
device_ID = ''
periodType = 'Week'
currentPeriod = pd.to_datetime('today').normalize()
searched_Data = data

#Declaring the file path, this differs with eacht computer, should be the path of the server folder
filepath = 'C:/Users/simon/Documents/GitHub/GUI-Heat-Tracing/'

#Declaring all the needed variables for the plots
data_A = np.array([])
data_B = np.array([])
data_C = np.array([])
data_D = np.array([])
deviceID_A = ' '
deviceID_B = ' '
deviceID_C = ' '
deviceID_D = ' '
timedates_A = np.array([])
timedates_B = np.array([])
timedates_C = np.array([])
timedates_D = np.array([])

#declaring all the variables for the table
df = pd.read_csv("Dummy_data.csv", sep=",")
df_sorted=df.sort_values(by=['Device_ID'])
df_sorted.head(10)
styled = style_table(df_sorted)
dfi.export(styled,"mytable.png", max_rows=200)
sortOn = 'Device_ID'


/* @author  Bart Withaar
 * @date    25-5-2021
 * @brief   Creating the plot design in this function
 * @param   dataframe: the given dataframe which needs to be styled
 * @note    None
 * @todo    Nothing
 * @retval  The styled dataframe
 */

In [202]:
def style_table(dataframe):
    dataframe = dataframe.head(10)
    dataframe = dataframe.style.set_table_styles(
           [{
               'selector': 'th',
               'props': [
                   ('font-size', '30pt'),
                   ('color', '#660066')]
           },
           {
               'selector': 'td',
               'props': [
                   ('font-size', '25pt'),
                   ('color', 'black')]
           }
           ])

    dataframe.hide_index()
    return dataframe  

/* @author  Bart Withaar
 * @date    22-5-2021
 * @brief   Updating the table with correct information
 * @param   df_filter, the incomming filtered dataframe
 * @param   strValue, the string value of the search bar
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [203]:
def update_table(df_filter, strValue):
    global sortOn
    if strValue == '':
        if sortOn == 'Temperature':
            df_filter = df_filter.sort_values('Date', ascending=False)
            df_filter = df_filter.sort_values(by=[sortOn], ascending=False)
        elif sortOn == 'Date':
            df_filter = df_filter.sort_values('Date', ascending=False)
        else:
            df_filter = df_filter.sort_values('Date', ascending=False)
            df_filter = df_filter.sort_values([sortOn], ascending=True)
        
        df_filter = df_filter.drop_duplicates(subset='Device_ID', keep='first')
    else:
        df_filter = df_filter.sort_values('Date', ascending=False)
        df_filter = df_filter.sort_values(by=[sortOn])
        
    #style the plot
    df_styled = style_table(df_filter)

    #export the data into an image
    os.remove("mytable.png")
    time.sleep(0.75)
    dfi.export(df_styled,"mytable.png", max_rows=20)

/* @author  Simon Balk, Joan den Otter
 * @date    24-5-2021
 * @brief   Getting the device data. Always call getData() before calling getTimeDates(), the information is updatet in the first function.
 * @param   deviceID: the device for which the data is needed
 * @note    None
 * @todo    Nothing
 * @retval  The temperature collum and the date collum from the csv file
 */

In [204]:
def getData(deviceID):
    global periodType
    global currentPeriod
    global searched_Data
    global data_from_csv
    data_from_csv = pd.read_csv('Dummy_data.CSV')
    data_from_csv['Date'] = pd.to_datetime(data_from_csv.Date)
    data_from_csv['Week'] = data.Date.dt.isocalendar().week
    data_from_csv['Month'] = data.Date.dt.month
    data_from_csv['Day'] = data.Date.dt.day

    device_ID = ('{}'.format(deviceID))
    

    if(periodType == 'Day'):
        timeDate = currentPeriod.day
    elif(periodType == 'Week'):
        timeDate = currentPeriod.week
    elif(periodType == 'Month'):
        timeDate = currentPeriod.month


    searched_Data = data_from_csv.loc[data_from_csv['Device_ID'] == device_ID, :]
    searched_Data = searched_Data.loc[data_from_csv[periodType] == timeDate, :]
    searched_Data = searched_Data.sort_values('Date', ascending=True)

    return searched_Data['Temperature']

#Never called first, always call getData before calling getTimeDates
def getTimeDates():
    global searched_Data

    return searched_Data['Date']



/* @author  Joan den Otter
 * @date    22-5-2021
 * @brief   Here the next and previous period get selected, this is based on the current period.
 * @param   None
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [205]:
def prevPeriod():    
    global periodType
    global currentPeriod
    if(periodType == 'Day'):
        currentPeriod = currentPeriod + relativedelta(days=-1)
    elif(periodType == 'Week'): 
        currentPeriod = currentPeriod + relativedelta(weeks=-1)
    elif(periodType == 'Month'):
        currentPeriod = currentPeriod + relativedelta(months=-1)   

def nextPeriod():    
    global periodType
    global currentPeriod
    if(periodType == 'Day'):
        currentPeriod = currentPeriod + relativedelta(days=+1)
    elif(periodType == 'Week'): 
        currentPeriod = currentPeriod + relativedelta(weeks=+1)
    elif(periodType == 'Month'):
        currentPeriod = currentPeriod + relativedelta(months=+1)  

/* @author  Simon Balk
 * @date    25-5-2021
 * @brief   This is where the plot is created and drawn, it does so based on the current deviceID's. If those id's are empty nothing will show
 * @param   None
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [206]:
def updatePlot():
    global deviceID_A
    global deviceID_B
    global deviceID_C
    global deviceID_D

    global data_A
    global data_B
    global data_C
    global data_D

    global timedates_A
    global timedates_B
    global timedates_C
    global timedates_D

    if deviceID_A != ' ':
        data_A = getData(deviceID_A)
        timedates_A = getTimeDates()
    if deviceID_B != ' ':
        data_B = getData(deviceID_B)
        timedates_B = getTimeDates()
    if deviceID_C != ' ':
        data_C = getData(deviceID_C)
        timedates_C = getTimeDates()
    if deviceID_D != ' ':
        data_D = getData(deviceID_D)
        timedates_D = getTimeDates()
    
    #Creating the plot
    if deviceID_B != ' ':
        if deviceID_C != ' ':
            #plot contains 3 or 4 graphs
            fig, ((graph_1,graph_3),(graph_2,graph_4)) = plt.subplots(2,2)
            if deviceID_A != ' ':
                graph_1.plot_date(timedates_A, data_A, linestyle = 'solid')
                graph_1.set_title(deviceID_A)
            if deviceID_B != ' ':
                graph_2.plot_date(timedates_B, data_B, linestyle = 'solid')
                graph_2.set_title(deviceID_B)
                graph_2.sharex(graph_1)
            if deviceID_C != ' ':
                graph_3.plot_date(timedates_C, data_C, linestyle = 'solid')
                graph_3.set_title(deviceID_C)
                #put in place here so dates will appear when there is only 3 graphs visable
                graph_4.sharex(graph_3)
            if deviceID_D != ' ':
                graph_4.plot_date(timedates_D, data_D, linestyle = 'solid')
                graph_4.set_title(deviceID_D)
        else:
            #plot contains 2 graphs
            fig, ((graph_1),(graph_2)) = plt.subplots(2,1)
            if deviceID_A != ' ':
                graph_1.plot_date(timedates_A, data_A, linestyle = 'solid')
                graph_1.set_title(deviceID_A)
            if deviceID_B != ' ':
                graph_2.plot_date(timedates_B, data_B, linestyle = 'solid')
                graph_2.set_title(deviceID_B)
                graph_2.sharex(graph_1)
    else:
        #plot only contains one graph
        fig, (graph_1) = plt.subplots(1,1)
        if deviceID_A != ' ':
            graph_1.plot_date(timedates_A, data_A, linestyle = 'solid')
            graph_1.set_title(deviceID_A)
    
    #Styling the plot
    fig.suptitle('Temperature')
    plt.style.use('seaborn')
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    #Showing the plot
    plt.draw()

/* @author  Simon Balk
 * @date    24-5-2021
 * @brief   In this function, lines will be added. This is only if the data is corresponding with a deviceID
 * @param   device_ID: the deviceID which the user has given as input
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [207]:
def addLine(device_ID):
    global deviceID_A
    global deviceID_B
    global deviceID_C
    global deviceID_D
    
    tempData = getData(device_ID)
    #Setting the incomming data in the correct variable
    if  tempData.size != 0:
        if deviceID_A == ' ':
            deviceID_A = str(device_ID)
        elif deviceID_B == ' ':
            deviceID_B = str(device_ID)
        elif deviceID_C == ' ':
            deviceID_C = str(device_ID)
        elif deviceID_D == ' ':
            deviceID_D = str(device_ID)
        else:
            print('Cannot add more graphs')
    #Updating the plot after adding a new device
    updatePlot()

/* @author  Simon Balk
 * @date    24-5-2021
 * @brief   This function remove(s) the selected graph/graphs
 * @param   graph: the selected graph which should be removed, if it is x, all graphs are removed.
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [208]:
def removeGraph(graph):
    global data_A
    global data_B
    global data_C
    global data_D
    global deviceID_A
    global deviceID_B
    global deviceID_C
    global deviceID_D
    global timedates_A
    global timedates_B
    global timedates_C
    global timedates_D
    
    if graph == 'A':
        #Graph A getting removed
        data_A = data_B
        data_B = data_C
        data_C = data_D
        data_D = np.array([])
        deviceID_A = deviceID_B
        deviceID_B = deviceID_C
        deviceID_C = deviceID_D
        deviceID_D = ' '
        timedates_A = timedates_B
        timedates_B = timedates_C
        timedates_C = timedates_D
        timedates_D = np.array([])
    elif graph == 'B':
        #Graph B getting removed
        data_B = data_C
        data_C = data_D
        data_D = np.array([])
        deviceID_B = deviceID_C
        deviceID_C = deviceID_D
        deviceID_D = ' '
        timedates_B = timedates_C
        timedates_C = timedates_D
        timedates_D = np.array([])
    elif graph == 'C':
        #Graph C getting removed
        data_C = data_D
        data_D = np.array([])
        deviceID_C = deviceID_D
        deviceID_D = ' '
        timedates_C = timedates_D
        timedates_D = np.array([])
    elif graph == 'D':
        #Graph D getting removed
        data_D = np.array([])
        deviceID_D = ' '
        timedates_D = np.array([])
    elif graph == 'X':
        #All graphs getting removed
        data_A = np.array([])
        data_B = np.array([])
        data_C = np.array([])
        data_D = np.array([])
        deviceID_A = ' '
        deviceID_B = ' '
        deviceID_C = ' '
        deviceID_D = ' '
        timedates_A = np.array([])
        timedates_B = np.array([])
        timedates_C = np.array([])
        timedates_D = np.array([])       
    #Calling the updatePlot() to generate a new plot
    updatePlot()

/* @author  Simon Balk
 * @date    21-5-2021
 * @brief   The imports for the anvil-uplink server and connecting it to the GUI
 * @note    None
 * @todo    Nothing
 * @retval  Nothing
 */

In [209]:
#pip install anvil-uplink
import anvil.server
import anvil.media
import anvil.mpl_util

anvil.server.connect("YYR43HPYADI257LCFSUHXF6C-E37CHCQWPQTFV5FH")

/* @author  Simon Balk, Joan den Otter
 * @date    25-5-2021
 * @brief   These funtions are for showing and updating the graph in the GUI
 * @param   input_ID: the id which the user has typed into the search bar
 * @param   graph: the graph/graphs which should be removed from the plot
 * @param   periodType: for setting the period to Day/Week/Month
 * @note    None
 * @todo    Nothing
 * @retval  A media object which will be shown in the GUI on the client side
 */

In [210]:
#Update the plot
@anvil.server.callable
def make_plot():
    updatePlot()
    return anvil.mpl_util.plot_image()

#Add the input_ID to the plot if it fits
@anvil.server.callable
def search_deviceID(input_ID):
    input_ID = str(input_ID)
    addLine(input_ID)

#Remove the selected graph from the plot
@anvil.server.callable
def removeGraphFromPlot(graph):
    removeGraph(str(graph))

#Change the period
@anvil.server.callable
def changePeriod(periodTypeInput):
    global periodType
    periodType = str(periodTypeInput)

#Select the next period for current selected period
@anvil.server.callable
def next():
    nextPeriod()

#Select the previous period for current selected period
@anvil.server.callable
def previous():
    prevPeriod()

#Change the time of the graphs to the current time
@anvil.server.callable
def setTimeToday():
    global currentPeriod
    currentPeriod = pd.to_datetime('today').normalize()

#Updating the data with the data from the csv file
@anvil.server.callable
def updateData():
    global data
    data_from_csv = pd.read_csv('Dummy_data.CSV')
    data_from_csv['Date'] = pd.to_datetime(data_from_csv.Date)
    data_from_csv['Week'] = data_from_csv.Date.dt.isocalendar().week
    data_from_csv['Month'] = data_from_csv.Date.dt.month
    data_from_csv['Day'] = data_from_csv.Date.dt.day

/* @author  Bart Withaar
 * @date    25-5-2021
 * @brief   These funtions are for updating the table on the main page of the app
 * @param   searchBarValue: contains the input of the search bar, this is coverted to a string at the beginning of the functions
 * @param   sortValue: this value corresponds with the pressed button in the GUI
 * @note    None
 * @todo    Nothing
 * @retval  A media object which will be shown in the GUI on the client side
 */

In [211]:
#Updating the plot with search bar value
@anvil.server.callable
def handle_submit(searchBarValue):
    searchBarValue = str(searchBarValue)
    df_filter=df[df.apply(lambda row: row.astype(str).str.contains(searchBarValue).any(), axis=1)]
    update_table(df_filter,searchBarValue)
    media_object = anvil.media.from_file(filepath + 'mytable.png')
    return media_object

#Updating the plot with search bar value and button pressed
@anvil.server.callable
def handle_buttonSubmit(searchBarValue, sortValue):
    searchBarValue = str(searchBarValue)
    global sortOn
    sortOn = sortValue
    df_filter=df[df.apply(lambda row: row.astype(str).str.contains(searchBarValue).any(), axis=1)]
    update_table(df_filter,searchBarValue)
    media_object = anvil.media.from_file(filepath + 'mytable.png')
    return media_object

#retrieving the plot for the first time
@anvil.server.callable
def get_df():
    media_object = anvil.media.from_file(filepath + 'mytable.png')
    return media_object